# TV Script Generation
In this notebook, we generate our own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using a multi-RNN cell built with TensorFlow.  The dataset is the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  


In [1]:
import helper
data_dir = './data/simpsons/full_history.txt'

text = helper.load_data(data_dir)
text = text[81:]

In [2]:
text[0:500]

"Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.\nLisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?\nMiss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?\nLisa Simpson: That life is worth living.\nEdna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, (SOUR) just in case any of you"

## Explore the Data
View different parts of the data.

In [3]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 136594
Number of scenes: 1
Average number of sentences in each scene: 158234.0
Number of lines: 158235
Average number of words in each line: 11.663614244636143

The sentences 0 to 10:
Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.
Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?
Miss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?
Lisa Simpson: That life is worth living.
Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, (SOUR) just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?
Martin Prince: (HOARSE WHISPER) I don't think there's anything left to say.
Edna Krabappel-Flanders: Bart?
Bart Simpson: Victory party under the slide!
(Apartment Building: Ext. apa

## Preprocessing Functions
Implement a couple of preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, two dictionaries are created:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # done: Implement Function
#     scenes = text.split('\n\n')
#     sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
#     words = []
#     # punctuations will be dealt with later
#     for sentence in sentences:
#         words.append(sentence.split())
    
    #comment: Note that "text" here is a different data type.  It is not one long string as it is earlier in this notebook.  It's already a list of words.
    vocab = sorted(set(text))
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = {ii: word for ii, word in enumerate(vocab)}
    return vocab_to_int, int_to_vocab


/home/thojo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Tokenize Punctuation
Splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

The function `token_lookup` will return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Creating a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Gotta make sure we don't use a token that could be confused as a word. Instead of using the token "dash", using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # done: Implement Function
    punct = {'.':'||Period||',
             ',':'||Comma||',
             '\"':'||Double_Quotation||',
             ';':'||Semi_Colon||',
             '!':'||Exclamation_Mark||',
             '?':'||Question_Mark||',
             '(':'||Left_Parentheses||',
             ')':'||Right_Parentheses||',
             '--':'||Dash||',
             '\n':'||Newline||',
             }
    return punct

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
The preprocessed data has been saved to disk.

In [7]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.10.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # done: Implement Function
    input_ = tf.placeholder( tf.int32, [None, None], name = 'input')
    targets_ = tf.placeholder( tf.int32, [None, None], name = 'targets')
    learning_rate = tf.placeholder( tf.float32, name = 'learning_rate')
    return input_, targets_, learning_rate

# tests.test_get_inputs(get_inputs)

### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def build_cell(num_units, keep_prob=0.8):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) #adding dropout
    return drop    

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs (TJ: dimension of the hidden state of the LSTM cell)
    :return: Tuple (cell, initialize state)
    """
    # done: Implement Function
    
    lstm_layers = 2
    ### Stack up multiple LSTM layers, for deep learning
#     cell = tf.contrib.rnn.MultiRNNCell([ lstm ]*lstm_layers) 
#     lstm = tf.contrib.rnn.BasicLSTMCell( rnn_size )   
#     no dropout.  If desired, add it like this and define cell as [drop]*lstm_layers
#     drop = tf.contrib.rnn.DropoutWrapper( lstm, output_keep_prob=keep_prob)
#     this is what I first had (works for TensorFlow 1.0.  For later versions, do as below i.e., [cell]*lstm_layers doesn't work)

    cell=tf.contrib.rnn.MultiRNNCell( [build_cell(rnn_size) for _ in range(lstm_layers)])   
      
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state') #gotamist: why was this necessary? There are other scripts (Sentiment_RNN for example), where the initial state has been set without a name
    return cell, initial_state

# tests.test_get_init_cell(get_init_cell)

### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """

    embedding = tf.Variable( tf.random_uniform(( vocab_size, embed_dim), -1, 1) )
    embed = tf.nn.embedding_lookup( embedding, input_data)
    return embed

# tests.test_get_embed(get_embed)

### Build RNN
  Time to use the cell created in `get_init_cell()` functionto create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [14]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # done: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype='float32')
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state

# tests.test_build_rnn(build_rnn)

### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [15]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data, vocab_size, embed_dim)    
    outputs, final_state = build_rnn(cell, embed)
#     logits = tf.layers.dense(outputs, units=vocab_size, activation=None) #trying the line below recommended by reviewer (includes initialization)
    logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev= 0.1),biases_initializer=tf.zeros_initializer())

    #     logits=tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    # note: both lines are legit for logits. fully_connected, behind the scenes, just calls tf.layers.dense
    return logits, final_state

# tests.test_build_nn(build_nn)

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If there isn't enough data to fill the last batch with enough data, drop the last batch.

In [17]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # done: Implement Function
    # I am assuming int_text will be a list rather than a 1-d array
    words_per_batch = batch_size * seq_length 
    n_batches = len(int_text) // words_per_batch
    x = np.array( int_text[:n_batches*words_per_batch] )# inputs
    y=np.zeros_like(x, dtype = x.dtype) # targets
    y[:-1], y[-1:] = x[1:],x[:1] 
    
    xarr = x.reshape((batch_size, -1))
    yarr = y.reshape((batch_size, -1))
    batches = np.zeros((n_batches, 2, batch_size, seq_length), dtype=int )
    batch_idx = 0
   
    for n in range(0, xarr.shape[1], seq_length):
        input_batch = xarr[:, n:n+seq_length]
        target_batch = yarr[:, n:n+seq_length]
        this_batch=np.array([input_batch, target_batch])
        batches[ batch_idx,:,:,:]=this_batch
        batch_idx += 1

    return batches

# tests.test_get_batches(get_batches)

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [18]:
# Number of Epochs
num_epochs = 25 #around 50 epochs were enough to get the loss under 1.0 with seq_length of 200. Loss goes down to 0.12 with 100 epochs.
# Batch Size
batch_size = 16
# RNN Size
rnn_size = 256
# Embedding Dimension Size

embed_dim = 300
# Sequence Length
seq_length = 12 # had 200 at first, but this is an improvement suggested by the reviewer (corresponding to the average sequence length in the training data of 11.5 words)
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 1000

save_dir = './save'

### Build the Graph
Build the graph using the neural network implemented above

In [34]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data. 

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/13372   train_loss = 10.922
Epoch   0 Batch 1000/13372   train_loss = 5.093
Epoch   0 Batch 2000/13372   train_loss = 5.123
Epoch   0 Batch 3000/13372   train_loss = 5.306
Epoch   0 Batch 4000/13372   train_loss = 4.531
Epoch   0 Batch 5000/13372   train_loss = 4.511
Epoch   0 Batch 6000/13372   train_loss = 4.176
Epoch   0 Batch 7000/13372   train_loss = 5.096
Epoch   0 Batch 8000/13372   train_loss = 5.015
Epoch   0 Batch 9000/13372   train_loss = 4.582
Epoch   0 Batch 10000/13372   train_loss = 4.460
Epoch   0 Batch 11000/13372   train_loss = 4.304
Epoch   0 Batch 12000/13372   train_loss = 4.470
Epoch   0 Batch 13000/13372   train_loss = 4.670
Epoch   1 Batch  628/13372   train_loss = 4.382
Epoch   1 Batch 1628/13372   train_loss = 4.199
Epoch   1 Batch 2628/13372   train_loss = 3.889
Epoch   1 Batch 3628/13372   train_loss = 4.878
Epoch   1 Batch 4628/13372   train_loss = 4.781
Epoch   1 Batch 5628/13372   train_loss = 4.274
Epoch   1 Batch 6628/13372   train_

Epoch  12 Batch 9536/13372   train_loss = 4.148
Epoch  12 Batch 10536/13372   train_loss = 3.301
Epoch  12 Batch 11536/13372   train_loss = 3.672
Epoch  12 Batch 12536/13372   train_loss = 3.865
Epoch  13 Batch  164/13372   train_loss = 3.763
Epoch  13 Batch 1164/13372   train_loss = 3.707
Epoch  13 Batch 2164/13372   train_loss = 3.682
Epoch  13 Batch 3164/13372   train_loss = 3.980
Epoch  13 Batch 4164/13372   train_loss = 3.497
Epoch  13 Batch 5164/13372   train_loss = 3.606
Epoch  13 Batch 6164/13372   train_loss = 4.056
Epoch  13 Batch 7164/13372   train_loss = 3.890
Epoch  13 Batch 8164/13372   train_loss = 3.478
Epoch  13 Batch 9164/13372   train_loss = 3.671
Epoch  13 Batch 10164/13372   train_loss = 3.624
Epoch  13 Batch 11164/13372   train_loss = 4.274
Epoch  13 Batch 12164/13372   train_loss = 3.496
Epoch  13 Batch 13164/13372   train_loss = 3.964
Epoch  14 Batch  792/13372   train_loss = 4.221
Epoch  14 Batch 1792/13372   train_loss = 3.827
Epoch  14 Batch 2792/13372   trai

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [36]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [37]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [20]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # done: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

# tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [19]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """

    # note: it would be less noisy to choose among words with the top 5 probabilities.

    p = probabilities / np.sum(probabilities)
    idx=np.random.choice(list(int_to_vocab.keys()), p=p)
    return int_to_vocab[idx]

# tests.test_pick_word(pick_word)

In [40]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [41]:
gen_length = 1000
# homer_simpson, moe_szyslak, or Barney_Gumble
# prime_word = 'moe_szyslak'
prime_word = 'lisa'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
#     gen_sentences = [prime_word + ':']
    gen_sentences = [prime_word]
    
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
lisa simpson: trust us, i went in and the guts home is to proud of all.
host:(chuckles) hey boys, i'm a winner on this.(starting tear)
homer simpson:(worried grunt)
homer simpson: how did you see that?
lenny leonard: oh well, now, there's another drink way to let me in, you don't know anything.
homer simpson: i'll save page to the life of james madison. which one of us do are you ominous that?
adult lisa: it hides it for mad like that.
(courtroom: int. courtroom - continuous)
that time we have a week over.
adult lisa: well dad, that's right.
lisa simpson:(sobs)(singing) take your rap look / hysterical) mrs. little heck, see? me and i'm going back to sleep.
bart simpson: sings changing our vinegar?
bart simpson: hey you're pity!
marge simpson:(startled yell) continuous)
homer simpson: um...
marge simpson: i don't know if it died again.
little boy:(annoyed) faster?
bergstrom: well, they know.
selma bouvier: you have a disease.
homer simpso

## Note 
Note that the parts of the script can be nonsensical.  But reading the real Simpsons scripts without visuals has a similar feeling.
